In [1]:
from __future__ import annotations

import sys
from pathlib import Path

import pandas as pd


def _find_scripts_dir() -> Path:
    """Find the folder that contains percentiles.py (web/backend/scripts)."""
    cwd = Path.cwd().resolve()
    candidates = [cwd] + list(cwd.parents)
    for base in candidates[:10]:
        p1 = base / "web" / "backend" / "scripts" / "percentiles.py"
        if p1.exists():
            return p1.parent
        p2 = base / "backend" / "scripts" / "percentiles.py"
        if p2.exists():
            return p2.parent
        p3 = base / "percentiles.py"
        if p3.exists():
            return p3.parent
    raise FileNotFoundError("Could not locate percentiles.py from current working directory.")


scripts_dir = _find_scripts_dir()
if str(scripts_dir) not in sys.path:
    sys.path.insert(0, str(scripts_dir))

from percentiles import get_percentiles  # noqa: E402


def _write_table_pdf(*, df: pd.DataFrame, pdf_path: Path, title: str, rows_per_page: int = 28) -> None:
    """Write a (potentially multi-page) PDF containing a wide table."""
    from matplotlib.backends.backend_pdf import PdfPages
    import matplotlib.image as mpimg
    import matplotlib.pyplot as plt

    brand_name = "Track Insights"
    brand_url = "trackinsights.org"

    # Translucent background logo on every page
    report_dir = pdf_path.parent
    logo_path = report_dir / "logo_transparentBG (1).png"
    logo_img = None
    if logo_path.exists():
        logo_img = mpimg.imread(str(logo_path))

    def _add_logo(fig, *, alpha: float, box: tuple[float, float, float, float]) -> None:
        if logo_img is None:
            return
        bg_ax = fig.add_axes(list(box), zorder=0)
        bg_ax.imshow(logo_img, alpha=alpha)
        bg_ax.axis("off")

    pdf_path.parent.mkdir(parents=True, exist_ok=True)

    with PdfPages(pdf_path) as pdf:
        total_rows = len(df)
        for start in range(0, total_rows, rows_per_page):
            end = min(start + rows_per_page, total_rows)
            chunk = df.iloc[start:end]

            fig, ax = plt.subplots(figsize=(17, 11))

            # Watermark behind the table
            _add_logo(fig, alpha=0.06, box=(0.28, 0.12, 0.44, 0.76))

            ax.axis("off")
            fig.suptitle(f"{title} (rows {start + 1}-{end} of {total_rows})", fontsize=14)

            table = ax.table(
                cellText=chunk.fillna("").values,
                colLabels=list(chunk.columns),
                loc="center",
                cellLoc="center",
            )
            table.auto_set_font_size(False)
            table.set_fontsize(7)
            table.scale(1.0, 1.25)

            # Branding footer (URL on every page)
            fig.text(0.01, 0.01, brand_name, ha="left", va="bottom", fontsize=9, color="#333333")
            fig.text(0.99, 0.01, brand_url, ha="right", va="bottom", fontsize=9, color="#333333")

            fig.tight_layout(rect=[0, 0.03, 1, 0.95])
            pdf.savefig(fig)
            plt.close(fig)


def _write_eventwise_pdf(*, wide_df: pd.DataFrame, percentiles: tuple[int, ...], pdf_path: Path) -> None:
    """Write a user-friendly PDF: title page + one page per event, percentiles listed top-down."""
    from matplotlib.backends.backend_pdf import PdfPages
    import matplotlib.image as mpimg
    import matplotlib.pyplot as plt

    brand_name = "Track Insights"
    brand_url = "trackinsights.org"

    # Page geometry (portrait / vertical)
    page_w_in = 8.5
    page_h_in = 11.0
    margin_in = 1.0
    table_gap_below_event_in = 1.0

    # Assets
    report_dir = pdf_path.parent
    logo_path = report_dir / "logo_transparentBG (1).png"
    logo_img = None
    if logo_path.exists():
        logo_img = mpimg.imread(str(logo_path))

    def _add_logo(fig, *, alpha: float, box: tuple[float, float, float, float]) -> None:
        if logo_img is None:
            return
        bg_ax = fig.add_axes(list(box), zorder=0)
        bg_ax.imshow(logo_img, alpha=alpha)
        bg_ax.axis("off")

    def _add_footer(fig, *, page_num: int | None, total_pages: int | None) -> None:
        # Keep footer inside bottom margin.
        y = (margin_in / 2.0) / page_h_in

        fig.text(0.12, y, brand_name, ha="left", va="center", fontsize=10, color="#333333")
        fig.text(0.88, y, brand_url, ha="right", va="center", fontsize=10, color="#333333")

        if page_num is not None and total_pages is not None:
            fig.text(
                0.5,
                y,
                f"page {page_num} / {total_pages}",
                ha="center",
                va="center",
                fontsize=10,
                color="#333333",
            )

    def _bold_table_headers_and_row_labels(table) -> None:
        # Header row is row 0 when using colLabels.
        for (r, c), cell in table.get_celld().items():
            if r == 0:
                cell.set_text_props(weight="bold")
            if c == 0 and r > 0:
                cell.set_text_props(weight="bold")

    # Build event list first so we can consistently page-number.
    events_to_render: list[tuple[str, list[object], list[object]]] = []
    for _, row in wide_df.iterrows():
        event = row["Event"]
        boys_vals = [row.get(f"Boys P{p}") for p in percentiles]
        girls_vals = [row.get(f"Girls P{p}") for p in percentiles]

        # Skip events with no data for either gender
        if all(pd.isna(v) or v == "" for v in boys_vals) and all(pd.isna(v) or v == "" for v in girls_vals):
            continue

        events_to_render.append((str(event), boys_vals, girls_vals))

    # The user asked for page _ / 17. Title page is unnumbered; event pages are 1..17.
    displayed_total_pages = 17
    if len(events_to_render) != displayed_total_pages:
        print(
            f"Warning: rendering {len(events_to_render)} event pages, "
            f"but numbering will display / {displayed_total_pages}."
        )

    pdf_path.parent.mkdir(parents=True, exist_ok=True)

    with PdfPages(pdf_path) as pdf:
        # --- Title page (no page number) ---
        fig = plt.figure(figsize=(page_w_in, page_h_in))

        # Make the logo fill more of the title page
        _add_logo(fig, alpha=0.35, box=(0.05, 0.12, 0.90, 0.76))

        ax = fig.add_axes([0, 0, 1, 1], zorder=1)
        ax.axis("off")

        fig.text(0.5, 0.83, brand_name, ha="center", va="center", fontsize=34, weight="bold")
        fig.text(0.5, 0.78, brand_url, ha="center", va="center", fontsize=14)

        fig.text(0.5, 0.66, "Percentiles Report", ha="center", va="center", fontsize=26, weight="bold")
        fig.text(0.5, 0.61, "Percentiles by 10 (Boys / Girls)", ha="center", va="center", fontsize=14)

        _add_footer(fig, page_num=None, total_pages=None)

        pdf.savefig(fig)
        plt.close(fig)

        # --- Event pages ---
        for i, (event, boys_vals, girls_vals) in enumerate(events_to_render, start=1):
            table_df = pd.DataFrame(
                {
                    "Percentile": [f"{p}%" for p in percentiles],
                    "Boys": ["" if pd.isna(v) else v for v in boys_vals],
                    "Girls": ["" if pd.isna(v) else v for v in girls_vals],
                }
            )

            fig = plt.figure(figsize=(page_w_in, page_h_in))

            # Translucent background watermark (more transparent than title page)
            _add_logo(fig, alpha=0.08, box=(0.22, 0.18, 0.56, 0.64))

            ax = fig.add_axes([0, 0, 1, 1], zorder=1)
            ax.axis("off")

            # Event name: inside 1" margins
            event_y = (page_h_in - margin_in) / page_h_in
            fig.text(0.5, event_y, event, ha="center", va="top", fontsize=18, weight="bold")

            # Table: keep 1" margins; start 1" below the event name
            x0 = margin_in / page_w_in
            y0 = margin_in / page_h_in
            table_top = (page_h_in - margin_in - table_gap_below_event_in) / page_h_in
            table_bbox = [x0, y0, 1.0 - (2 * x0), max(0.0, table_top - y0)]

            table = ax.table(
                cellText=table_df.values,
                colLabels=list(table_df.columns),
                cellLoc="center",
                bbox=table_bbox,
            )
            table.auto_set_font_size(False)
            table.set_fontsize(12)
            table.scale(1.0, 1.6)

            _bold_table_headers_and_row_labels(table)
            _add_footer(fig, page_num=i, total_pages=displayed_total_pages)

            pdf.savefig(fig)
            plt.close(fig)


# Percentiles by 10: 10,20,...,90
percentiles = tuple(range(10, 100, 10))

# Run query (all events, both genders, all years/meet types by default)
df_long = get_percentiles(percentiles=percentiles)

# Wide format: one row per event, Boys columns then Girls columns
wide = df_long.pivot(index="Event", columns="Gender", values=list(percentiles))
wide = wide.swaplevel(0, 1, axis=1)  # (Gender, Percentile)
wide = wide.reindex(columns=pd.MultiIndex.from_product([["Boys", "Girls"], percentiles]))
wide.columns = [f"{gender} P{p}" for gender in ("Boys", "Girls") for p in percentiles]
wide = wide.reset_index()

# More natural event ordering (falls back to alphabetical for unknown events)
event_order = [
    "100 Meters",
    "200 Meters",
    "400 Meters",
    "800 Meters",
    "1600 Meters",
    "3200 Meters",
    "100 Hurdles",
    "110 Hurdles",
    "300 Hurdles",
    "4 x 100 Relay",
    "4 x 400 Relay",
    "4 x 800 Relay",
    "High Jump",
    "Long Jump",
    "Pole Vault",
    "Shot Put",
    "Discus",
    "Triple Jump",
    "Javelin",
]
order_index = {name: i for i, name in enumerate(event_order)}
wide["_order"] = wide["Event"].map(order_index).fillna(10_000).astype(int)
wide = wide.sort_values(["_order", "Event"]).drop(columns=["_order"]).reset_index(drop=True)

# Output folder
report_dir = scripts_dir / "percentiles report"
report_dir.mkdir(parents=True, exist_ok=True)

# Output paths
out_tsv = report_dir / "percentiles_by10.tsv"
out_pdf_boys = report_dir / "percentiles_by10_boys.pdf"
out_pdf_girls = report_dir / "percentiles_by10_girls.pdf"
out_pdf_readable = report_dir / "percentiles_by10_readable.pdf"

# Save TSV (best for copy/paste into a spreadsheet)
wide.to_csv(out_tsv, sep="\t", index=False)
print(wide.to_csv(sep="\t", index=False))
print(f"Wrote TSV: {out_tsv}")

# Wide-table PDFs (one per gender) - exclude events that don't exist for that gender
boys_value_cols = [f"Boys P{p}" for p in percentiles]
girls_value_cols = [f"Girls P{p}" for p in percentiles]

boys_df = wide[["Event"] + boys_value_cols].dropna(subset=boys_value_cols, how="all")
girls_df = wide[["Event"] + girls_value_cols].dropna(subset=girls_value_cols, how="all")

_write_table_pdf(df=boys_df, pdf_path=out_pdf_boys, title="Percentiles by 10 - Boys")
_write_table_pdf(df=girls_df, pdf_path=out_pdf_girls, title="Percentiles by 10 - Girls")

# User-friendly PDF: title page + one page per event, percentiles top-down, Boys/Girls side-by-side
_write_eventwise_pdf(wide_df=wide, percentiles=percentiles, pdf_path=out_pdf_readable)

print(f"Wrote PDF (Boys): {out_pdf_boys}")
print(f"Wrote PDF (Girls): {out_pdf_girls}")
print(f"Wrote PDF (Readable): {out_pdf_readable}")


Event	Boys P10	Boys P20	Boys P30	Boys P40	Boys P50	Boys P60	Boys P70	Boys P80	Boys P90	Girls P10	Girls P20	Girls P30	Girls P40	Girls P50	Girls P60	Girls P70	Girls P80	Girls P90
100 Meters	12.94	12.54	12.29	12.08	11.92	11.74	11.58	11.35	11.10	15.33	14.71	14.33	14.07	13.85	13.62	13.37	13.09	12.73
200 Meters	26.68	25.82	25.26	24.76	24.40	24.02	23.65	23.18	22.59	32.22	30.98	30.11	29.42	28.87	28.33	27.76	27.21	26.39
400 Meters	1:01.82	59.01	57.40	56.40	55.39	54.50	53.58	52.38	51.15	1:16.15	1:12.75	1:10.63	1:08.72	1:07.28	1:05.77	1:04.58	1:02.77	1:00.54
800 Meters	2:34.76	2:26.28	2:21.47	2:17.10	2:13.83	2:10.32	2:06.44	2:03.30	1:59.67	3:09.89	2:59.73	2:53.07	2:48.34	2:43.54	2:39.06	2:34.66	2:29.41	2:23.66
1600 Meters	5:48.06	5:29.18	5:17.85	5:08.05	5:00.50	4:54.06	4:45.63	4:37.44	4:28.44	7:05.13	6:43.11	6:28.74	6:18.57	6:07.38	5:56.34	5:45.72	5:33.43	5:20.97
3200 Meters	12:43.50	12:01.98	11:37.12	11:13.46	10:53.49	10:37.71	10:20.68	10:01.72	9:41.67	15:16.16	14:32.61	13:58.35	13:31.57	13:07.7

C:\Users\justi\AppData\Local\Temp\ipykernel_8500\1222126986.py:86: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  fig.tight_layout(rect=[0, 0.03, 1, 0.95])
C:\Users\justi\AppData\Local\Temp\ipykernel_8500\1222126986.py:86: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  fig.tight_layout(rect=[0, 0.03, 1, 0.95])


Wrote PDF (Boys): C:\Users\justi\OneDrive\Desktop\CS Seminar\trackinsights-1\web\backend\scripts\percentiles report\percentiles_by10_boys.pdf
Wrote PDF (Girls): C:\Users\justi\OneDrive\Desktop\CS Seminar\trackinsights-1\web\backend\scripts\percentiles report\percentiles_by10_girls.pdf
Wrote PDF (Readable): C:\Users\justi\OneDrive\Desktop\CS Seminar\trackinsights-1\web\backend\scripts\percentiles report\percentiles_by10_readable.pdf
